# Advanced Big Data: Spark SQL and DataFrames

------------------------------------
# Author:
- Mohamed NIANG, Data Scientist at Paris-Saclay University
- Penda TOURE, Data Scientist at Paris-Saclay University

# Teacher:
- Youcef Sebiat, Data Engineer / Scientist at CMAP / Lead Data Scientist at l'X École Polytechnique
- e-mail address: youcef.sebiat@polytechnique.edu
-----------------------------------

### 1. Basic manipulations
First, let's create some users:

In [3]:
from datetime import date

# Let's first create some users:
col_names = ["first_name", "last_name", "birth_date", "gender", "country"]
users = [
  ("Alice", "Jones", date(1981, 4, 15), "female", "Canada"),
  ("John", "Doe", date(1951, 1, 21), "male", "USA"),
  ("Barbara", "May", date(1951, 9, 1), "female", "Australia"),
  ("James", "Smith", date(1975, 7, 12), "male", "United Kingdom"),
  ("Gerrard", "Dupont", date(1968, 5, 9), "male", "France"),
  ("Amanda", "B.", date(1988, 12, 16), "female", "New Zeland")
]

users_df = spark.createDataFrame(users, col_names)
display(users_df) # Only works in Databricks. Elswehere, use "df.show()" or "df.toPandas()"

first_name,last_name,birth_date,gender,country
Alice,Jones,1981-04-15,female,Canada
John,Doe,1951-01-21,male,USA
Barbara,May,1951-09-01,female,Australia
James,Smith,1975-07-12,male,United Kingdom
Gerrard,Dupont,1968-05-09,male,France
Amanda,B.,1988-12-16,female,New Zeland


Now it's your turn, create more users (at least 3, with different names) and add to the initial users, saving the result in a new variable.

In [5]:
new_users_df = spark.createDataFrame([
  ("Mohamed", "Niang", date(1995, 9, 24), "male", "Senegal"),
  ("Fernanda", "Tchouacheu", date(1997, 5, 12), "female", "Cameroun"),
  ("Penda", "Toure", date(1996, 11, 13), "female", "Senegal"),
])
all_users_df = users_df.union(new_users_df)
display(all_users_df) # or all_users_df.show()

first_name,last_name,birth_date,gender,country
Alice,Jones,1981-04-15,female,Canada
John,Doe,1951-01-21,male,USA
Barbara,May,1951-09-01,female,Australia
James,Smith,1975-07-12,male,United Kingdom
Gerrard,Dupont,1968-05-09,male,France
Amanda,B.,1988-12-16,female,New Zeland
Mohamed,Niang,1995-09-24,male,Senegal
Fernanda,Tchouacheu,1997-05-12,female,Cameroun
Penda,Toure,1996-11-13,female,Senegal


Now, select only two columns and show the resulting DataFrame, without saving it into a variable.

In [7]:
all_users_df.select("first_name","last_name").show()

+----------+----------+
first_name| last_name|
+----------+----------+
 Alice| Jones|
 John| Doe|
 Barbara| May|
 James| Smith|
 Gerrard| Dupont|
 Amanda| B.|
 Mohamed| Niang|
 Fernanda|Tchouacheu|
 Penda| Toure|
+----------+----------+

Now, register your DataFrame as a table and select the same two columns with a SQL query string

In [9]:
query_string = "SELECT first_name, last_name FROM all_users_df_view"
all_users_df.createTempView('all_users_df_view') # creates a local temporary table accessible by a SQL query
spark.sql(query_string).show()

+----------+----------+
first_name| last_name|
+----------+----------+
 Alice| Jones|
 John| Doe|
 Barbara| May|
 James| Smith|
 Gerrard| Dupont|
 Amanda| B.|
 Mohamed| Niang|
 Fernanda|Tchouacheu|
 Penda| Toure|
+----------+----------+

Now we want to add an unique identifier for each user in the table. There are many strategies for that, and for our example we will use the string `{last_name}_{first_name}`

You can use `all_users_df` since your latest operation did not override its values. Add a new column called `user_id` to your DataFrame and save to a new variable.

**Hint:** The first place to look is in the [functions](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions) package

In [11]:
from pyspark.sql import functions as fn
from pyspark.sql.functions import concat, col, lit

users_with_id = all_users_df.withColumn('user_id', concat(col("first_name"), lit("_"), col("last_name")))
display(users_with_id)

first_name,last_name,birth_date,gender,country,user_id
Alice,Jones,1981-04-15,female,Canada,Alice_Jones
John,Doe,1951-01-21,male,USA,John_Doe
Barbara,May,1951-09-01,female,Australia,Barbara_May
James,Smith,1975-07-12,male,United Kingdom,James_Smith
Gerrard,Dupont,1968-05-09,male,France,Gerrard_Dupont
Amanda,B.,1988-12-16,female,New Zeland,Amanda_B.
Mohamed,Niang,1995-09-24,male,Senegal,Mohamed_Niang
Fernanda,Tchouacheu,1997-05-12,female,Cameroun,Fernanda_Tchouacheu
Penda,Toure,1996-11-13,female,Senegal,Penda_Toure


You can also do the same thing with an User Defined Function by passing a lambda, although it is not recommended when there is already a function in the `functions` package.

Add a new column called `user_id_udf` to the DataFrame, using an UDF that receives two parameters and concatenate them.

In [13]:
from pyspark.sql.types import StringType

concat_udf = fn.udf(lambda l,v: l + "_" + v, StringType())
users_with_id_udf = all_users_df.withColumn('user_id', concat_udf(all_users_df.first_name, all_users_df.last_name))
display(users_with_id_udf)

first_name,last_name,birth_date,gender,country,user_id
Alice,Jones,1981-04-15,female,Canada,Alice_Jones
John,Doe,1951-01-21,male,USA,John_Doe
Barbara,May,1951-09-01,female,Australia,Barbara_May
James,Smith,1975-07-12,male,United Kingdom,James_Smith
Gerrard,Dupont,1968-05-09,male,France,Gerrard_Dupont
Amanda,B.,1988-12-16,female,New Zeland,Amanda_B.
Mohamed,Niang,1995-09-24,male,Senegal,Mohamed_Niang
Fernanda,Tchouacheu,1997-05-12,female,Cameroun,Fernanda_Tchouacheu
Penda,Toure,1996-11-13,female,Senegal,Penda_Toure


Now, let's add another column called `age` with the computed age (in years) of each user, based on a given reference date, and save the resulting DataFrame into a new variable.

**Hint:** You can first compute the age in months, and then divide by 12. A final operation will probably be needed to get an integer number.

In [15]:
from pyspark.sql.functions import datediff

reference_date = date(2017, 12, 31)
age = datediff(lit(reference_date), all_users_df.birth_date)/365

users_with_age = all_users_df.withColumn('age', age.cast('int'))
display(users_with_age)

first_name,last_name,birth_date,gender,country,age
Alice,Jones,1981-04-15,female,Canada,36
John,Doe,1951-01-21,male,USA,66
Barbara,May,1951-09-01,female,Australia,66
James,Smith,1975-07-12,male,United Kingdom,42
Gerrard,Dupont,1968-05-09,male,France,49
Amanda,B.,1988-12-16,female,New Zeland,29
Mohamed,Niang,1995-09-24,male,Senegal,22
Fernanda,Tchouacheu,1997-05-12,female,Cameroun,20
Penda,Toure,1996-11-13,female,Senegal,21


Now, an analytical question: How many users of each gender who are more than 40 years old exist in this data? The solution must be a DataFrame with two columns: `gender` and `count` and two lines, one for each gender.

Bonus: Try to do your solution in a single chain (without intermediate variables)

**Hint:** You will need to filter and aggregate the data.

In [17]:
result = users_with_age.filter(users_with_age.age > 40).groupBy('gender').count()
display(result)

gender,count
female,1
male,3


### 2. Reading files, performing joins, and aggregating

For this section you will use some fake data of two datasets: `Users` and `Donations`. The data is provided in two CSV files *with header* and using *comma* as separator.

The `Users` dataset contains information about about the users. 

The `Donations` dataset contains information about Donations performed by those users.

The first task is to read these files into the appropriate DataFrames.

**Note:** You need to set the option "inferSchema" to true in order to have the columns in the correct types.

_The data for this section has been created using [Mockaroo](https://www.mockaroo.com/)_.

In [19]:
users_from_file = spark.read.format("csv") \
  .option("inferSchema", "True") \
  .option("header", "True") \
  .option("sep", ",") \
  .load("/FileStore/tables/users.csv")

donations = spark.read.format("csv") \
  .option("inferSchema", "True") \
  .option("header", "True") \
  .option("sep", ",") \
  .load("/FileStore/tables/donations.csv")

Now investigate the columns, contents and size of both datasets

In [21]:
# print the column names and types
users_from_file.printSchema()
donations.printSchema()

# print 5 elements of the datasets in a tabular format
users_from_file.select("*").limit(5).show()
donations.select("*").limit(5).show()

# print the number of lines of each dataset
print(users_from_file.count())
print(donations.count())

root
-- user_id: integer (nullable = true)
-- first_name: string (nullable = true)
-- last_name: string (nullable = true)
-- gender: string (nullable = true)
-- birth_date: timestamp (nullable = true)
-- country: string (nullable = true)

root
-- donation_id: integer (nullable = true)
-- user_id: integer (nullable = true)
-- donation_value: double (nullable = true)
-- date: timestamp (nullable = true)
-- time: string (nullable = true)
-- timestamp: timestamp (nullable = true)

+-------+----------+---------+------+-------------------+-------------+
user_id|first_name|last_name|gender| birth_date| country|
+-------+----------+---------+------+-------------------+-------------+
 1| Janifer| Lagadu|Female|1940-08-09 00:00:00|United States|
 2| Bruis| Danilov| Male|1966-05-28 00:00:00| Brazil|
 3| Omar| Spinozzi| Male|1977-08-28 00:00:00| China|
 4| Mag| Kennedy|Female|1963-11-11 00:00:00| China|
 5| Gerome| Lushey| Male|1990-06-08 00:00:00| Brazil|
+-------+----------+---------+------+-------------------+-------------+

+-----------+-------+--------------+-------------------+--------+-------------------+
donation_id|user_id|donation_value| date| time| timestamp|
+-----------+-------+--------------+-------------------+--------+-------------------+
 1| 63| 25.41|2017-03-06 00:00:00|18:28:20|2017-03-06 18:28:20|
 2| 53| 93.32|2017-03-15 00:00:00|15:15:09|2017-03-15 15:15:09|
 3| 74| 136.04|2017-10-29 00:00:00|20:36:23|2017-10-29 20:36:23|
 4| 94| 37.25|2017-05-16 00:00:00|12:33:58|2017-05-16 12:33:58|
 5| 83| 89.45|2017-05-09 00:00:00|22:45:44|2017-05-09 22:45:44|
+-----------+-------+--------------+-------------------+--------+-------------------+

100
1000

**Note:** If all the column types shown in the previous results are "string", you need to make sure you passed "inferSchema" as true when reading the CSV files before continuing.

Before using the data, we may want to add some information about the users. 

Add a column containing the age of each user.

In [23]:
from pyspark.sql.functions import datediff, substring

users_from_file = users_from_file.withColumn('age', (datediff(lit(reference_date), users_from_file.birth_date.substr(1,10))/365).cast('int'))
users_from_file.show()

+-------+----------+----------+------+-------------------+-------------+---+
user_id|first_name| last_name|gender| birth_date| country|age|
+-------+----------+----------+------+-------------------+-------------+---+
 1| Janifer| Lagadu|Female|1940-08-09 00:00:00|United States| 77|
 2| Bruis| Danilov| Male|1966-05-28 00:00:00| Brazil| 51|
 3| Omar| Spinozzi| Male|1977-08-28 00:00:00| China| 40|
 4| Mag| Kennedy|Female|1963-11-11 00:00:00| China| 54|
 5| Gerome| Lushey| Male|1990-06-08 00:00:00| Brazil| 27|
 6| Lazarus| Andrus| Male|1983-04-22 00:00:00| France| 34|
 7| Adora| Bartunek|Female|1971-06-04 00:00:00| France| 46|
 8| Gladys| Gulland|Female|1959-07-07 00:00:00| Brazil| 58|
 9| Pearline| Kitcher|Female|1944-06-01 00:00:00| China| 73|
 10| Jolie| Diggin|Female|1944-07-31 00:00:00| Nigeria| 73|
 11| Devlen| Devil| Male|1955-01-11 00:00:00|United States| 63|
 12| Sibel| Lewisham|Female|1982-11-05 00:00:00| France| 35|
 13| Paton| Brundrett| Male|1951-11-08 00:00:00| China| 66|
 14| Cody|Menichillo| Male|1958-08-11 00:00:00| China| 59|
 15| Brinna| Lannon|Female|1985-07-23 00:00:00| Brazil| 32|
 16| Abe| Grima| Male|1971-08-23 00:00:00| France| 46|
 17| Ginger| Lipp| Male|1963-07-16 00:00:00| Japan| 54|
 18| Braden| Jachimiak| Male|1996-05-10 00:00:00| China| 21|
 19| Sharl| Airy|Female|1941-09-08 00:00:00| Brazil| 76|
 20| Erv| Waumsley| Male|1960-07-21 00:00:00| China| 57|
+-------+----------+----------+------+-------------------+-------------+---+
only showing top 20 rows

Another useful information to have is the age **range** of each user. Using the `when` function, create the following 5 age ranges:
- "(0, 25]"
- "(25, 35]"
- "(35, 45]"
- "(45, 55]"
- "(55, ~]"

And add a new column to the users DataFrame, containing this information.

**Note:** When building logical operations with Spark DataFrames, it's better to be add parantheses. Example:
```python
df.select("name", "age").where( (df("age") > 20) & (df("age") <= 30) )
```

**Note 2:** If you are having problems with the `when` function, you can make an User Defined Function and do your logic in standard python.

In [25]:
from pyspark.sql import functions as fn

age_range = fn.when((users_from_file.age <= 25), '0-25')\
              .when(((users_from_file.age > 25) & (users_from_file.age <= 35)), '25-35')\
              .when(((users_from_file.age > 35) & (users_from_file.age <= 45)), '35-45')\
              .when(((users_from_file.age > 45) & (users_from_file.age <= 55)), '45-55')\
              .otherwise('55 and more')
                    
users_from_file = users_from_file.withColumn('age_range', age_range)
users_from_file.show()

+-------+----------+----------+------+-------------------+-------------+---+-----------+
user_id|first_name| last_name|gender| birth_date| country|age| age_range|
+-------+----------+----------+------+-------------------+-------------+---+-----------+
 1| Janifer| Lagadu|Female|1940-08-09 00:00:00|United States| 77|55 and more|
 2| Bruis| Danilov| Male|1966-05-28 00:00:00| Brazil| 51| 45-55|
 3| Omar| Spinozzi| Male|1977-08-28 00:00:00| China| 40| 35-45|
 4| Mag| Kennedy|Female|1963-11-11 00:00:00| China| 54| 45-55|
 5| Gerome| Lushey| Male|1990-06-08 00:00:00| Brazil| 27| 25-35|
 6| Lazarus| Andrus| Male|1983-04-22 00:00:00| France| 34| 25-35|
 7| Adora| Bartunek|Female|1971-06-04 00:00:00| France| 46| 45-55|
 8| Gladys| Gulland|Female|1959-07-07 00:00:00| Brazil| 58|55 and more|
 9| Pearline| Kitcher|Female|1944-06-01 00:00:00| China| 73|55 and more|
 10| Jolie| Diggin|Female|1944-07-31 00:00:00| Nigeria| 73|55 and more|
 11| Devlen| Devil| Male|1955-01-11 00:00:00|United States| 63|55 and more|
 12| Sibel| Lewisham|Female|1982-11-05 00:00:00| France| 35| 25-35|
 13| Paton| Brundrett| Male|1951-11-08 00:00:00| China| 66|55 and more|
 14| Cody|Menichillo| Male|1958-08-11 00:00:00| China| 59|55 and more|
 15| Brinna| Lannon|Female|1985-07-23 00:00:00| Brazil| 32| 25-35|
 16| Abe| Grima| Male|1971-08-23 00:00:00| France| 46| 45-55|
 17| Ginger| Lipp| Male|1963-07-16 00:00:00| Japan| 54| 45-55|
 18| Braden| Jachimiak| Male|1996-05-10 00:00:00| China| 21| 0-25|
 19| Sharl| Airy|Female|1941-09-08 00:00:00| Brazil| 76|55 and more|
 20| Erv| Waumsley| Male|1960-07-21 00:00:00| China| 57|55 and more|
+-------+----------+----------+------+-------------------+-------------+---+-----------+
only showing top 20 rows

Now that we have improved our users' DataFrame, the first analysis we want to make is the average donation performed by each gender. However, the gender information and the donation value are in different tables, so first we need to join them, using the `user_id` as joining key.

**Note:** Make sure you are not using the `users` DataFrame from the first part of the lab.

**Note 2:** For better performance, you can [broadcast](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.broadcast) the smaller dataset. But you should only do this when you are sure the DataFrame fits in the memory of the nodes.

In [27]:
joined_df = users_from_file.join(donations, 'user_id')
display(joined_df)

user_id,first_name,last_name,gender,birth_date,country,age,age_range,donation_id,donation_value,date,time,timestamp
63,Tabor,Bramer,Male,1983-05-29T00:00:00.000+0000,Brazil,34,25-35,1,25.41,2017-03-06T00:00:00.000+0000,18:28:20,2017-03-06T18:28:20.000+0000
53,Chris,Climie,Male,1970-06-02T00:00:00.000+0000,France,47,45-55,2,93.32,2017-03-15T00:00:00.000+0000,15:15:09,2017-03-15T15:15:09.000+0000
74,Livia,Lodemann,Female,1967-03-11T00:00:00.000+0000,United States,50,45-55,3,136.04,2017-10-29T00:00:00.000+0000,20:36:23,2017-10-29T20:36:23.000+0000
94,Kendre,Pankhurst.,Female,1941-05-28T00:00:00.000+0000,China,76,55 and more,4,37.25,2017-05-16T00:00:00.000+0000,12:33:58,2017-05-16T12:33:58.000+0000
83,Quincy,Marquess,Male,1997-08-13T00:00:00.000+0000,Brazil,20,0-25,5,89.45,2017-05-09T00:00:00.000+0000,22:45:44,2017-05-09T22:45:44.000+0000
80,Boothe,Endley,Male,1960-03-03T00:00:00.000+0000,China,57,55 and more,6,163.67,2017-07-25T00:00:00.000+0000,21:50:36,2017-07-25T21:50:36.000+0000
19,Sharl,Airy,Female,1941-09-08T00:00:00.000+0000,Brazil,76,55 and more,7,10.69,2017-11-15T00:00:00.000+0000,8:43:54,2017-11-15T08:43:54.000+0000
53,Chris,Climie,Male,1970-06-02T00:00:00.000+0000,France,47,45-55,8,135.09,2017-04-14T00:00:00.000+0000,7:43:37,2017-04-14T07:43:37.000+0000
46,Ethelyn,Raubenheim,Female,1953-07-19T00:00:00.000+0000,Brazil,64,55 and more,9,94.38,2017-03-07T00:00:00.000+0000,10:30:34,2017-03-07T10:30:34.000+0000
90,Farlee,O'Connell,Male,1985-05-15T00:00:00.000+0000,China,32,25-35,10,68.63,2017-02-02T00:00:00.000+0000,14:23:43,2017-02-02T14:23:43.000+0000


Now, use aggregation to find the the min, max and avg donation by gender.

In [29]:
donations_by_gender = joined_df.groupBy('gender').agg(
    fn.min('donation_value').alias('min of donation'),
    fn.max('donation_value').alias('max of donation'),
    fn.avg('donation_value').alias('average of donation'))
donations_by_gender.show()

+------+---------------+---------------+-------------------+
gender|min of donation|max of donation|average of donation|
+------+---------------+---------------+-------------------+
Female| 1.25| 199.87| 97.62667400881057|
 Male| 1.12| 199.93| 100.23606227106244|
+------+---------------+---------------+-------------------+

Now, make the necessary transformations and aggregations to answer to the following questions about the data. Note that some questions are only about the users, so make sure to use the smaller possible dataset when looking for your answers!

**Question 1:** 

a) What's the average, min and max age of the users? 

b) What's the average, min and max age of the users, by gender?

In [31]:
result_1a = users_from_file.agg(
    fn.avg('age').alias('average of age'),
    fn.min('age').alias('min of age'),
    fn.max('age').alias('max of age'))
result_1a.show()

result_1b = users_from_file.groupBy('gender').agg(
    fn.avg('age').alias('average of age'),
    fn.min('age').alias('min of age'),
    fn.max('age').alias('max of age'))
result_1b.show()

+--------------+----------+----------+
average of age|min of age|max of age|
+--------------+----------+----------+
 48.23| 19| 77|
+--------------+----------+----------+

+------+-----------------+----------+----------+
gender| average of age|min of age|max of age|
+------+-----------------+----------+----------+
Female|52.59183673469388| 19| 77|
 Male|44.03921568627451| 19| 75|
+------+-----------------+----------+----------+

**Question 2:**

a) How many distinct country origins exist in the data? Print a DataFrame listing them.

b) What are the top 5 countries with the most users? Print a DataFrame containing the name of the countries and the counts.

In [33]:
result_2a = users_from_file.select("country").distinct()
result_2a.show()

result_2b = users_from_file.groupBy('country').count().orderBy(fn.desc('count')).limit(5)
result_2b.show()

+-------------+
 country|
+-------------+
 Germany|
 France|
United States|
 China|
 Nigeria|
 Brazil|
 Japan|
 New Zealand|
+-------------+

+-------------+-----+
 country|count|
+-------------+-----+
 China| 54|
 Brazil| 19|
 France| 13|
United States| 5|
 Japan| 4|
+-------------+-----+

**Question 3:**

What's the number of donations average, min and max donations values by age range?

In [35]:
result_3 = joined_df.groupBy('age_range').agg(
    fn.avg('donation_value').alias('average of donations'),
    fn.min('donation_value').alias('min of donations'),
    fn.max('donation_value').alias('max of donations'))
result_3.show()

+-----------+--------------------+----------------+----------------+
 age_range|average of donations|min of donations|max of donations|
+-----------+--------------------+----------------+----------------+
 35-45| 98.2091366906474| 2.03| 199.32|
55 and more| 99.63908011869435| 1.36| 197.79|
 45-55| 102.5383009708738| 1.19| 199.93|
 0-25| 99.38162500000003| 4.33| 199.28|
 25-35| 95.58210084033615| 1.12| 199.87|
+-----------+--------------------+----------------+----------------+

**Question 4:**

a) What's the number of donations, average, min and max donation values by user location (country)?

b) What is the number of donations, average, min and max donation values by gender for each user location (contry)? (the resulting DataFrame must contain 5 columns: the gender of the user, their country and the 3 metrics)

In [37]:
result_4a = joined_df.groupBy('country').agg(
    fn.avg('donation_value').alias('average of donations'),
    fn.min('donation_value').alias('min of donations'),
    fn.max('donation_value').alias('max of donations'))
result_4a.show()

result_4b = joined_df.groupBy('gender','country').agg(
    fn.avg('donation_value').alias('average of donations'),
    fn.min('donation_value').alias('min of donations'),
    fn.max('donation_value').alias('max of donations'))
result_4b.show()

+-------------+--------------------+----------------+----------------+
 country|average of donations|min of donations|max of donations|
+-------------+--------------------+----------------+----------------+
 Germany| 95.64157894736842| 17.81| 180.48|
 France| 95.63199999999999| 5.79| 199.93|
United States| 108.3796| 1.36| 198.15|
 China| 97.62353463587931| 1.25| 199.32|
 Nigeria| 81.22600000000001| 14.47| 192.63|
 Brazil| 105.70655555555568| 1.12| 199.91|
 Japan| 95.63270270270273| 1.19| 199.76|
 New Zealand| 107.93| 12.98| 172.51|
+-------------+--------------------+----------------+----------------+

+------+-------------+--------------------+----------------+----------------+
gender| country|average of donations|min of donations|max of donations|
+------+-------------+--------------------+----------------+----------------+
Female| Japan| 116.16818181818181| 23.52| 192.32|
 Male| France| 95.74000000000001| 5.79| 199.93|
 Male| Germany| 95.64157894736842| 17.81| 180.48|
Female|United States| 124.63000000000001| 26.15| 195.98|
Female| New Zealand| 107.93| 12.98| 172.51|
Female| Nigeria| 81.22600000000001| 14.47| 192.63|
 Male|United States| 89.30304347826086| 1.36| 198.15|
 Male| Japan| 86.94461538461542| 1.19| 199.76|
Female| Brazil| 102.05333333333331| 1.49| 196.41|
 Male| China| 100.5750485436893| 1.47| 199.32|
Female| China| 94.0329133858268| 1.25| 198.88|
 Male| Brazil| 108.14203703703706| 1.12| 199.91|
Female| France| 95.52033898305086| 14.36| 199.87|
+------+-------------+--------------------+----------------+----------------+

**Question 5**

Which month of the year has the largest aggregated donation value?

**Hint**: you can use a function to extract the month from a date, then you can aggregate to find the total donation value.

In [39]:
from pyspark.sql.functions import *

result_5 = joined_df.groupBy(month(joined_df.date.substr(1,10)).alias('month')).agg(fn.sum('donation_value').alias('total donation')).orderBy(fn.desc('total donation'))
result_5.show()

+-----+------------------+
month| total donation|
+-----+------------------+
 5|10696.400000000001|
 12|10430.539999999997|
 9| 8971.71|
 11| 8530.99|
 6| 8354.05|
 8| 8344.849999999999|
 4| 8308.579999999998|
 10| 8287.28|
 7| 8151.220000000003|
 3|7646.9299999999985|
 1| 6752.159999999997|
 2| 4576.69|
+-----+------------------+

### 3. Window Functions

_This section uses the same data as the last one._

Window functions are very useful for gathering aggregated data without actually aggregating the DataFrame. They can also be used to find "previous" and "next" information for entities, such as an user. [This article](https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html) has a very nice explanation about the concept.

We want to find the users who donated less than a threshold and remove them from the donations dataset. Now, there are two ways of doing that:

1) Performing a traditional aggregation to find the users who donated less than the threshold, then filtering these users from the donations dataset, either with `where(not(df("user_id").isin(a_local_list)))` or with a join of type "anti-join".

2) Using window functions to add a new column with the aggregated donations per user, and then using a normal filter such as `where(aggregated_donation < threshold)`

Let's implement both and compare the complexity:

First, perform the traditional aggregation and find the users who donated less than 500 in total:

In [42]:
bad_users = joined_df.groupBy('user_id').agg(fn.sum('donation_value').alias('aggregated_donation'))
bad_users = bad_users.where(bad_users.aggregated_donation < 500)
bad_users.show()

+-------+-------------------+
user_id|aggregated_donation|
+-------+-------------------+
 85| 437.78999999999996|
 47| 351.84|
 96| 354.97|
 84| 329.46999999999997|
 87| 497.60999999999996|
 82| 291.73|
 73| 455.20000000000005|
 25| 413.09000000000003|
 42| 215.35|
 99| 370.99|
+-------+-------------------+

You should have found around 10 users. Now, perform an "anti-join" to remove those users from `joined_df`.

**Hint:** The `join` operation accepts a third argument which is the join type. The accepted values are: 'inner', 'outer', 'full', 'fullouter', 'leftouter', 'left', 'rightouter', 'right', 'leftsemi', 'leftanti', 'cross'.

In [44]:
good_donations = joined_df.join(bad_users, joined_df.user_id == bad_users.user_id, 'leftanti')
good_donations.count()

Out[134]: 955

Verify if the count of `good_donations` makes sense by performing a normal join to find the `bad_donations`.

In [46]:
bad_donations = joined_df.join(bad_users, joined_df.user_id == bad_users.user_id, 'inner')
bad_donations.count()

Out[136]: 45

If you done everything right, at this point `good_donations.count()` + `bad_donations.count()` = `joined_df.count()`.

But using the join approach can be very heavy and it requires multipe operations. For this kind of problems, Window Functions are better.

In [48]:
good_donations.count() + bad_donations.count() == joined_df.count()

Out[137]: True

The first step is to create your window specification over `user_id` by using partitionBy

In [50]:
from pyspark.sql import Window

window_spec = Window.partitionBy('user_id')

Then, you can use one of the window functions of the `pyspark.sql.functions` package, appling to the created window_spec by using the `over` method. 

**Hint:** If you are blocked, try looking at the [documentation](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.over) for the `over` method, or searching on StackOverflow.

In [52]:
new_column = fn.sum('donation_value').over(window_spec)

donations_with_total = joined_df.withColumn('aggregated_donation', new_column)
donations_with_total.show()

+-------+----------+---------+------+-------------------+-------+---+-----------+-----------+--------------+-------------------+--------+-------------------+-------------------+
user_id|first_name|last_name|gender| birth_date|country|age| age_range|donation_id|donation_value| date| time| timestamp|aggregated_donation|
+-------+----------+---------+------+-------------------+-------+---+-----------+-----------+--------------+-------------------+--------+-------------------+-------------------+
 31| Pebrook| Gearty| Male|1993-03-31 00:00:00| China| 24| 0-25| 292| 42.9|2017-05-04 00:00:00|20:28:36|2017-05-04 20:28:36| 746.76|
 31| Pebrook| Gearty| Male|1993-03-31 00:00:00| China| 24| 0-25| 311| 155.49|2017-05-03 00:00:00| 8:58:01|2017-05-03 08:58:01| 746.76|
 31| Pebrook| Gearty| Male|1993-03-31 00:00:00| China| 24| 0-25| 341| 35.93|2016-12-10 00:00:00| 5:04:28|2016-12-10 05:04:28| 746.76|
 31| Pebrook| Gearty| Male|1993-03-31 00:00:00| China| 24| 0-25| 529| 128.41|2017-08-04 00:00:00|23:23:02|2017-08-04 23:23:02| 746.76|
 31| Pebrook| Gearty| Male|1993-03-31 00:00:00| China| 24| 0-25| 546| 9.56|2017-09-10 00:00:00|10:38:37|2017-09-10 10:38:37| 746.76|
 31| Pebrook| Gearty| Male|1993-03-31 00:00:00| China| 24| 0-25| 727| 9.5|2017-08-26 00:00:00|14:43:38|2017-08-26 14:43:38| 746.76|
 31| Pebrook| Gearty| Male|1993-03-31 00:00:00| China| 24| 0-25| 757| 85.05|2017-01-17 00:00:00|14:37:28|2017-01-17 14:37:28| 746.76|
 31| Pebrook| Gearty| Male|1993-03-31 00:00:00| China| 24| 0-25| 784| 29.96|2017-03-22 00:00:00| 6:41:37|2017-03-22 06:41:37| 746.76|
 31| Pebrook| Gearty| Male|1993-03-31 00:00:00| China| 24| 0-25| 804| 35.6|2017-03-20 00:00:00| 2:46:47|2017-03-20 02:46:47| 746.76|
 31| Pebrook| Gearty| Male|1993-03-31 00:00:00| China| 24| 0-25| 841| 15.08|2017-06-28 00:00:00| 2:42:01|2017-06-28 02:42:01| 746.76|
 31| Pebrook| Gearty| Male|1993-03-31 00:00:00| China| 24| 0-25| 883| 199.28|2016-11-29 00:00:00|13:00:43|2016-11-29 13:00:43| 746.76|
 85| Ashien| Biaggi|Female|1945-07-01 00:00:00|Nigeria| 72|55 and more| 39| 33.28|2016-12-24 00:00:00| 3:19:52|2016-12-24 03:19:52| 437.78999999999996|
 85| Ashien| Biaggi|Female|1945-07-01 00:00:00|Nigeria| 72|55 and more| 104| 115.24|2017-11-09 00:00:00| 4:22:45|2017-11-09 04:22:45| 437.78999999999996|
 85| Ashien| Biaggi|Female|1945-07-01 00:00:00|Nigeria| 72|55 and more| 186| 49.37|2017-05-25 00:00:00|11:04:56|2017-05-25 11:04:56| 437.78999999999996|
 85| Ashien| Biaggi|Female|1945-07-01 00:00:00|Nigeria| 72|55 and more| 472| 106.73|2017-05-24 00:00:00| 7:57:06|2017-05-24 07:57:06| 437.78999999999996|
 85| Ashien| Biaggi|Female|1945-07-01 00:00:00|Nigeria| 72|55 and more| 596| 133.17|2017-08-19 00:00:00| 0:14:39|2017-08-19 00:14:39| 437.78999999999996|
 65| Sayres| Halls| Male|1974-06-07 00:00:00| Japan| 43| 35-45| 41| 70.01|2017-07-17 00:00:00|18:18:43|2017-07-17 18:18:43| 804.3499999999999|
 65| Sayres| Halls| Male|1974-06-07 00:00:00| Japan| 43| 35-45| 154| 174.36|2017-07-11 00:00:00| 5:49:22|2017-07-11 05:49:22| 804.3499999999999|
 65| Sayres| Halls| Male|1974-06-07 00:00:00| Japan| 43| 35-45| 265| 42.11|2017-07-21 00:00:00| 5:38:01|2017-07-21 05:38:01| 804.3499999999999|
 65| Sayres| Halls| Male|1974-06-07 00:00:00| Japan| 43| 35-45| 282| 56.96|2016-12-03 00:00:00|17:34:36|2016-12-03 17:34:36| 804.3499999999999|
+-------+----------+---------+------+-------------------+-------+---+-----------+-----------+--------------+-------------------+--------+-------------------+-------------------+
only showing top 20 rows

And now you can just filter on the `total_donated_by_user` column:

In [54]:
good_donations_wf = donations_with_total.filter(donations_with_total.aggregated_donation >= 500)
good_donations_wf.count()

Out[140]: 955

If you done everything right, you should obtain `good_donations_wf.count()` = `good_donations.count()`

In [56]:
good_donations_wf.count() == good_donations.count()

Out[141]: True

Window functions also can be useful to find the "next" and "previous" operations by using `functions.lead` and `functions.lag`. In our example, we can use it to find the date interval between two donations by a specific user.

For this kind of Window function, the window specification must be ordered by the date. So, create a new window specification partitioned by the `user_id` and ordered by the donation timestamp.

In [58]:
ordered_window = Window.partitionBy('user_id').orderBy(joined_df.timestamp.substr(1,10))

Now use `functions.lag().over()` to add a column with the timestamp of the previous donation of the same user. Then, inspect the result to see what it looks like.

In [60]:
new_column = fn.lag(joined_df.timestamp.substr(1,10), 1).over(ordered_window)

donations_with_lag = good_donations.withColumn('previous_donation', new_column)
donations_with_lag.orderBy("user_id", "timestamp").show()

+-------+----------+---------+------+-------------------+-------------+---+-----------+-----------+--------------+-------------------+--------+-------------------+-----------------+
user_id|first_name|last_name|gender| birth_date| country|age| age_range|donation_id|donation_value| date| time| timestamp|previous_donation|
+-------+----------+---------+------+-------------------+-------------+---+-----------+-----------+--------------+-------------------+--------+-------------------+-----------------+
 1| Janifer| Lagadu|Female|1940-08-09 00:00:00|United States| 77|55 and more| 560| 121.82|2016-12-03 00:00:00| 3:34:03|2016-12-03 03:34:03| null|
 1| Janifer| Lagadu|Female|1940-08-09 00:00:00|United States| 77|55 and more| 486| 188.37|2017-02-01 00:00:00|20:20:44|2017-02-01 20:20:44| 2016-12-03|
 1| Janifer| Lagadu|Female|1940-08-09 00:00:00|United States| 77|55 and more| 145| 95.88|2017-05-13 00:00:00| 5:27:25|2017-05-13 05:27:25| 2017-02-01|
 1| Janifer| Lagadu|Female|1940-08-09 00:00:00|United States| 77|55 and more| 564| 85.05|2017-08-11 00:00:00| 0:19:06|2017-08-11 00:19:06| 2017-05-13|
 1| Janifer| Lagadu|Female|1940-08-09 00:00:00|United States| 77|55 and more| 561| 195.98|2017-09-01 00:00:00| 4:04:17|2017-09-01 04:04:17| 2017-08-11|
 1| Janifer| Lagadu|Female|1940-08-09 00:00:00|United States| 77|55 and more| 264| 150.51|2017-09-21 00:00:00| 4:36:37|2017-09-21 04:36:37| 2017-09-01|
 1| Janifer| Lagadu|Female|1940-08-09 00:00:00|United States| 77|55 and more| 317| 51.55|2017-10-26 00:00:00| 1:55:37|2017-10-26 01:55:37| 2017-09-21|
 2| Bruis| Danilov| Male|1966-05-28 00:00:00| Brazil| 51| 45-55| 36| 197.01|2016-11-28 00:00:00|15:41:01|2016-11-28 15:41:01| null|
 2| Bruis| Danilov| Male|1966-05-28 00:00:00| Brazil| 51| 45-55| 524| 91.76|2017-05-07 00:00:00|14:32:42|2017-05-07 14:32:42| 2016-11-28|
 2| Bruis| Danilov| Male|1966-05-28 00:00:00| Brazil| 51| 45-55| 460| 65.68|2017-05-11 00:00:00| 3:32:57|2017-05-11 03:32:57| 2017-05-07|
 2| Bruis| Danilov| Male|1966-05-28 00:00:00| Brazil| 51| 45-55| 316| 190.49|2017-05-16 00:00:00| 8:41:48|2017-05-16 08:41:48| 2017-05-11|
 2| Bruis| Danilov| Male|1966-05-28 00:00:00| Brazil| 51| 45-55| 54| 118.02|2017-05-22 00:00:00| 9:40:27|2017-05-22 09:40:27| 2017-05-16|
 2| Bruis| Danilov| Male|1966-05-28 00:00:00| Brazil| 51| 45-55| 138| 173.28|2017-06-29 00:00:00|10:03:02|2017-06-29 10:03:02| 2017-05-22|
 2| Bruis| Danilov| Male|1966-05-28 00:00:00| Brazil| 51| 45-55| 642| 133.67|2017-08-26 00:00:00|18:39:30|2017-08-26 18:39:30| 2017-06-29|
 2| Bruis| Danilov| Male|1966-05-28 00:00:00| Brazil| 51| 45-55| 628| 172.69|2017-09-09 00:00:00|21:20:39|2017-09-09 21:20:39| 2017-08-26|
 2| Bruis| Danilov| Male|1966-05-28 00:00:00| Brazil| 51| 45-55| 222| 154.52|2017-09-10 00:00:00| 5:54:41|2017-09-10 05:54:41| 2017-09-09|
 3| Omar| Spinozzi| Male|1977-08-28 00:00:00| China| 40| 35-45| 566| 125.9|2016-12-01 00:00:00|13:49:19|2016-12-01 13:49:19| null|
 3| Omar| Spinozzi| Male|1977-08-28 00:00:00| China| 40| 35-45| 177| 152.98|2016-12-31 00:00:00|21:14:54|2016-12-31 21:14:54| 2016-12-01|
 3| Omar| Spinozzi| Male|1977-08-28 00:00:00| China| 40| 35-45| 268| 58.17|2017-01-21 00:00:00| 8:42:12|2017-01-21 08:42:12| 2016-12-31|
 3| Omar| Spinozzi| Male|1977-08-28 00:00:00| China| 40| 35-45| 387| 74.61|2017-02-17 00:00:00| 3:04:01|2017-02-17 03:04:01| 2017-01-21|
+-------+----------+---------+------+-------------------+-------------+---+-----------+-----------+--------------+-------------------+--------+-------------------+-----------------+
only showing top 20 rows

Finally, compute the average time it took for each user between two of their consecutive donations (in days), and print the 5 users with the smallest averages. The result must include at least the users' id, last name and birth date, as well as the computed average.

In [62]:
new_column2 = fn.lead(joined_df.timestamp.substr(1,10), 1).over(ordered_window)
donations_with_lag = donations_with_lag.withColumn('next_donation', new_column2)
donations_with_lag = donations_with_lag.withColumn('day_duration', (datediff(lit(donations_with_lag.next_donation), donations_with_lag.previous_donation)).cast('int'))
result = donations_with_lag.groupBy('user_id').agg(fn.avg('day_duration').alias('average_duration_in_days')).orderBy(fn.asc('average_duration_in_days'))
donations_with_lag = result.join(donations_with_lag,"user_id",'left').select('user_id','last_name','birth_date','average_duration_in_days')

users_average_between_donations = donations_with_lag.orderBy(fn.asc('average_duration_in_days')).drop_duplicates()
users_average_between_donations.show(5)

+-------+---------+-------------------+------------------------+
user_id|last_name| birth_date|average_duration_in_days|
+-------+---------+-------------------+------------------------+
 67| Hambatch|1954-04-14 00:00:00| 28.0|
 78| Sherston|1965-07-23 00:00:00| 33.54545454545455|
 6| Andrus|1983-04-22 00:00:00| 34.8125|
 91| Kildale|1979-01-25 00:00:00| 36.705882352941174|
 63| Bramer|1983-05-29 00:00:00| 38.76470588235294|
+-------+---------+-------------------+------------------------+
only showing top 5 rows

Congratulations, you have finished this notebook!